In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.4.1


In [ ]:
!python

Python 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
>>> 
KeyboardInterrupt
^C


In [ ]:
!ls 'drive/My Drive/Enzyme'

inputs	models	notebooks  source


In [ ]:
# Import Libraries
import sys

sys.path.insert(0,'drive/My Drive/Enzyme')


import numpy as np
import random
import pandas as pd
import os
import tensorflow as tf
import pickle
import keras


from numpy.random import seed
from keras.models import Sequential,Model
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Input,concatenate, Bidirectional, SpatialDropout1D, GlobalAveragePooling1D,Reshape,Conv2D, MaxPool2D
from keras.layers import Add, Activation,LayerNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.regularizers import l1, l2




In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.119.225.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.119.225.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# build Tokenizer
df = pd.read_csv('drive/My Drive/Enzyme/inputs/Train.csv')
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(df.SEQUENCE.values.tolist())

In [ ]:
# Save Tokenizer
with open('drive/MyDrive/Enzyme/models/tokenizerndr.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# Fix seed
seed(1)
tf.random.set_seed(seed=2)

EPOCHS = 30
batch_size = 4096
max_len = 300
lr = 0.001

# Training Function
def train(tokenizer,fold_):

    df = pd.read_csv('drive/My Drive/Enzyme/inputs/stratified_5_fold.csv')
    # dataframes for train and val 
    
    train_df = df[df.kfold != fold_].reset_index(drop=True)

    val_df = df[df.kfold == fold_].reset_index(drop=True)

    
    #tokenize train and val

    xtrain = tokenizer.texts_to_sequences(train_df.SEQUENCE.values)
    xval = tokenizer.texts_to_sequences(val_df.SEQUENCE.values)

    # pad sequence
    xtrain = tf.keras.preprocessing.sequence.pad_sequences(xtrain,padding='post',truncating='post',maxlen=max_len)
    xval = tf.keras.preprocessing.sequence.pad_sequences(xval,padding='post',truncating='post',maxlen=max_len)

    # extract labels from dataframes
    ytrain = train_df.loc[:,['LABEL']].values
    yval = val_df.loc[:,['LABEL']].values

    # one hot encoding
    ytrain_enc = np_utils.to_categorical(ytrain)
    yval_enc = np_utils.to_categorical(yval)

    word_index = tokenizer.word_index

    # printing shapes of train and val 
    print(xtrain.shape)
    print(xval.shape)

    print(ytrain_enc.shape)
    print(yval_enc.shape)

    print(len(word_index))

    # Define Model: Training on TPU
    
    with tpu_strategy.scope():
        
        sequence_input = Input(shape=(max_len,))

        embedding_layer1 = Embedding(input_dim=len(word_index)+1 ,
                                output_dim=128,
                            input_length=max_len)

        x_emb1 = embedding_layer1(sequence_input)
        x = SpatialDropout1D(0.2)(x_emb1)

        x = Bidirectional(GRU(300,return_sequences=True))(x)
        x = Bidirectional(GRU(300,return_sequences=True))(x)
        x = Conv1D(300, kernel_size = 7, padding = "same",activation='relu', kernel_initializer = "he_uniform")(x)
        avg_pool1 = GlobalAveragePooling1D()(x)
        max_pool1 = GlobalMaxPooling1D()(x)     
        
        x = concatenate([avg_pool1, max_pool1])
        x = Dense(1024, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.8)(x)
        preds = Dense(20, activation='softmax')(x)
        
        model = Model(sequence_input, preds)
        

        model.summary()
      
        model.compile(loss='categorical_crossentropy',steps_per_execution = 50,
                      optimizer= tf.keras.optimizers.Adam(learning_rate=lr)
                      ,metrics=['accuracy'])

    # Fit the model with early stopping,save_best and reduce on plateau callbacks
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
    save_best = ModelCheckpoint(f"drive/My Drive/Enzyme/models/model_{fold_}.h5", save_best_only=True, monitor='val_loss', mode='min')
    rl = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_delta=1e-4, mode='min')



    model.fit(xtrain, y=ytrain_enc, batch_size=batch_size, epochs=EPOCHS, 
          verbose=1, validation_data=(xval, yval_enc), callbacks=[earlystop,save_best,rl])
    
    #return tokenizer
    
   

In [ ]:
# Load saved Tokenizer and start training
with open('drive/MyDrive/Enzyme/models/tokenizerndr.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

for i in range(5):
  train(tokenizer,fold_=i)





In [ ]:
# Inference 

fold = 5
num_classes = 20
# Load Tokenizer
with open('drive/MyDrive/Enzyme/models/tokenizerndr.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load and tokenize test data
test_df = pd.read_csv('drive/MyDrive/Enzyme/inputs/Test.csv')

xtest = tokenizer.texts_to_sequences(test_df.SEQUENCE.values)
xtest = tf.keras.preprocessing.sequence.pad_sequences(xtest,padding='post',truncating='post',maxlen=300)
final_preds = []

for i in range(fold):
  loaded_model = tf.keras.models.load_model(f"drive/My Drive/Enzyme/models/model_{i}.h5")
  preds = loaded_model.predict(xtest)
  print(preds.shape)
  final_preds.append(preds)


   
pred_0 = np.concatenate(final_preds,axis=0)
print(pred_0.shape)
n_pred = pred_0.reshape(fold,test_df.shape[0],num_classes)
avg = np.mean(n_pred,axis=0)
y_classes = np.argmax(avg,axis=1)
print(y_classes)

(253146, 20)
(253146, 20)
(253146, 20)
(253146, 20)
(253146, 20)
(1265730, 20)
[ 7  2  2 ... 19  0  7]


In [ ]:
# groud truth labels given by label_encoder
'''
ground = ['class0' 'class1' 'class10' 'class11' 'class12' 'class13' 'class14'
 'class15' 'class16' 'class17' 'class18' 'class19' 'class2' 'class3'
 'class4' 'class5' 'class6' 'class7' 'class8' 'class9']'''

idx2class = {0:'class0',1:'class1',2:'class10',3:'class11',4:'class12',5:'class13',6:'class14',7:'class15',
             8:'class16',9:'class17',10:'class18',11:'class19',12:'class2',13:'class3',
             14:'class4',15:'class5',16:'class6',17:'class7',18:'class8',19:'class9'}

sub = pd.DataFrame()
sub["SEQUENCE_ID"] = test_df["SEQUENCE_ID"]
sub["LABEL"] = y_classes
sub['LABEL'].replace(idx2class, inplace=True)
sub.head()
sub.to_csv('drive/MyDrive/Enzyme/inputs/submission.csv', index=False)



In [ ]:
sub = pd.read_csv('drive/MyDrive/Enzyme/inputs/subm.csv')
sub.head()

In [ ]:
# Pseudo_labeling

# We need to add the test sequences to the submission file in order to add it to our training data, then retrain the model

'''
ground = ['class0' 'class1' 'class10' 'class11' 'class12' 'class13' 'class14'
 'class15' 'class16' 'class17' 'class18' 'class19' 'class2' 'class3'
 'class4' 'class5' 'class6' 'class7' 'class8' 'class9']'''

idx2class = {0:'class0',1:'class1',2:'class10',3:'class11',4:'class12',5:'class13',6:'class14',7:'class15',
             8:'class16',9:'class17',10:'class18',11:'class19',12:'class2',13:'class3',
             14:'class4',15:'class5',16:'class6',17:'class7',18:'class8',19:'class9'}

sub = pd.DataFrame()
sub["SEQUENCE_ID"] = test_df["SEQUENCE_ID"]
# Adding test sequences to submission
sub["SEQUENCE"] = test_df["SEQUENCE"]
sub["LABEL"] = y_classes
sub['LABEL'].replace(idx2class, inplace=True)
sub.head()
sub.to_csv('drive/MyDrive/Enzyme/inputs/submission.csv', index=False)

In [ ]:
# open original training file
# Concatenate training file with submission file

train = pd.read_csv('drive/MyDrive/Enzyme/inputs/Train.csv')
sub = pd.read_csv('drive/MyDrive/Enzyme/inputs/submission.csv')
new_train = train.drop('CREATURE',axis=1)

combined_csv = pd.concat([new_train,sub])
#export to csv
combined_csv.to_csv( "drive/MyDrive/Enzyme/inputs/new_train.csv", index=False)


In [ ]:
# The new training file will go under the same procedure of creating folds, tokenization, and training 
